**Import Libaraies**

In [1]:
# Importing libraries 
import numpy as np 
import tensorflow 
from tensorflow import keras 
import matplotlib as plt 
import seaborn 
import glob 
import os
from os import listdir 

from keras.preprocessing.image import ImageDataGenerator

import skimage 
from skimage.io import imread_collection
from skimage.io import imshow_collection
from skimage.io import concatenate_images 
from matplotlib import pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras import Sequential 

from PIL import Image

We have to set the directory based on the folder where you saved the  dataset

In [2]:
for dirname, _, filenames in os.walk('C:\\Users\\0MEN\\Desktop\\sub\\resizedto224\\'):
    print (dirname)

C:\Users\0MEN\Desktop\sub\resizedto224\
C:\Users\0MEN\Desktop\sub\resizedto224\pred
C:\Users\0MEN\Desktop\sub\resizedto224\pred\pred
C:\Users\0MEN\Desktop\sub\resizedto224\Test
C:\Users\0MEN\Desktop\sub\resizedto224\Test\actinic keratosis
C:\Users\0MEN\Desktop\sub\resizedto224\Test\basal cell carcinoma
C:\Users\0MEN\Desktop\sub\resizedto224\Test\dermatofibroma
C:\Users\0MEN\Desktop\sub\resizedto224\Test\melanoma
C:\Users\0MEN\Desktop\sub\resizedto224\Test\nevus
C:\Users\0MEN\Desktop\sub\resizedto224\Test\pigmented benign keratosis
C:\Users\0MEN\Desktop\sub\resizedto224\Test\seborrheic keratosis
C:\Users\0MEN\Desktop\sub\resizedto224\Train
C:\Users\0MEN\Desktop\sub\resizedto224\Train\actinic keratosis
C:\Users\0MEN\Desktop\sub\resizedto224\Train\basal cell carcinoma
C:\Users\0MEN\Desktop\sub\resizedto224\Train\dermatofibroma
C:\Users\0MEN\Desktop\sub\resizedto224\Train\melanoma
C:\Users\0MEN\Desktop\sub\resizedto224\Train\nevus
C:\Users\0MEN\Desktop\sub\resizedto224\Train\pigmented beni

We have to set the directory based on the folder where you saved the  dataset(train and test)

In [3]:
# Importing the data and preprocessing 
from keras.preprocessing.image import ImageDataGenerator 


# set the directory based on the folder where you saved dataset
train_data_dir = "C:\\Users\\0MEN\\Desktop\\sub\\resizedto224\\Train\\"
test_data_dir = "C:\\Users\\0MEN\\Desktop\\sub\\resizedto224\\Test\\"
image_generator = ImageDataGenerator(rescale=1/255, validation_split=0.2)    

train_data = image_generator.flow_from_directory(batch_size=32,
                                                 directory=train_data_dir,
                                                 shuffle=True,
                                                 target_size=(227,227), 
                                                 subset="training",
                                                 class_mode='categorical')

val_data = image_generator.flow_from_directory(batch_size=32,
                                                 directory=train_data_dir,
                                                 shuffle=True,
                                                 target_size=(227,227), 
                                                 subset="validation",
                                                 class_mode='categorical')
test_gen = ImageDataGenerator(rescale=1./255)
test_data = test_gen.flow_from_directory(test_data_dir,
                                       target_size=(227,227), batch_size=32, shuffle=False)


print(train_data.classes)
print(val_data.classes)
print(train_data.class_indices)
print(test_data.classes)
class_names = np.unique(train_data.classes)

Found 1538 images belonging to 7 classes.
Found 381 images belonging to 7 classes.
Found 99 images belonging to 7 classes.
[0 0 0 ... 6 6 6]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6]
{'actinic keratosis': 0, 'basal cell carcinoma': 1, 'dermatofibroma': 2, 'melanoma': 

We have to change the value in Dense according to number of class..

In [4]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
#change the value in Dense according to number of class..
    keras.layers.Dense(7, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 384)       8

In [5]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
history = model.fit(train_data, epochs=5, batch_size=32, validation_data = val_data)

Epoch 1/5
49/49 [==============================] - 88s 1s/step - loss: 17.5776 - accuracy: 0.2906 - val_loss: 19.2086 - val_accuracy: 0.1864
Epoch 2/5
49/49 [==============================] - 45s 910ms/step - loss: 3.3840 - accuracy: 0.3635 - val_loss: 2.9432 - val_accuracy: 0.1759
Epoch 3/5
49/49 [==============================] - 45s 910ms/step - loss: 2.3334 - accuracy: 0.4207 - val_loss: 2.4180 - val_accuracy: 0.1391
Epoch 4/5
49/49 [==============================] - 45s 909ms/step - loss: 1.9803 - accuracy: 0.4350 - val_loss: 1.8100 - val_accuracy: 0.2625
Epoch 5/5
49/49 [==============================] - 45s 910ms/step - loss: 1.8217 - accuracy: 0.4649 - val_loss: 1.8076 - val_accuracy: 0.2835


In [7]:
test_loss, test_acc = model.evaluate((test_data))

4/4 [==============================] - 3s 731ms/step - loss: 1.9573 - accuracy: 0.2626


In [8]:
model.save('C:\\Users\\0MEN\\Desktop\\sub\\alexnet\\intel_alexnet_dec21.h5')